In [ ]:
# !pip install arxiv

Load wikipdia tool,Arxiv Tool and make a tool but firstly in this case load the data,chunks,embedd ,store in db,make retriver from db then finally create a tool from it>>make a final tool by combining alll thse three in a single tool,just by passing it as a list

Load chat llm model like llama2 from ollama>>import chatprompt template from langchain hub

Make Agents by passsing llm,prompt and tool as parameter in create_openai_tools_agent()
then create Agent Executor by passing the tools and agents in AgentExecutor()
Finally invoke by Agent Executor to get the reponses from the desired tools

### Wikipedia Tool

In [ ]:
from langchain_community.tools import WikipediaQueryRun  # Premade LangChain Tool
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [3]:
wiki.run("What is Machine Learning?")

'Page: Machine learning\nSummary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen'

What it is:
WikipediaQueryRun is a LangChain Tool that knows how to use WikipediaAPIWrapper to search Wikipedia and return results.

A Tool in LangChain = a function wrapped with some metadata (like its name, description, input type) so an agent or you can call it easily.

You are connecting your code with Wikipedia using LangChain’s built-in tools.

Here’s what each part is really doing in plain words:

WikipediaAPIWrapper(...) → this is like a helper that knows how to go to Wikipedia, search for something, and bring back the content.

top_k_results=1 → bring back only the top 1 most relevant page.

doc_content_chars_max=200 → don’t bring the whole article, just the first 200 characters (to save space).

WikipediaQueryRun(api_wrapper=api_wrapper) → this wraps the helper into a tool that LangChain (or you) can call directly. Think of it like a ready-made function: “search Wikipedia and return text.”

wiki.name → this just gives the name of the tool, which is useful if you plug it into an agent (so the agent knows which tool it is using).

👉 In very simple terms:
You’ve created a mini search engine for Wikipedia inside LangChain. When you call it, it will:

take your question (e.g., “What is Python programming?”)

search Wikipedia

return the first 200 characters of the top matching page.

### Created Tool from Langsmith Website

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents[:1],OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever


USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\vishal.puri\AppData\Local\Temp\ipykernel_20212\2444006978.py:9: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb=FAISS.from_documents(documents[:1],OllamaEmbeddings())


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002416E9677D0>, search_kwargs={})

##### Load>>Chunks>>Embedd>>Store in vector db>>make retriever

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")
retriever_tool.name

'langsmith_search'

In [8]:
retriever_tool.run("what is Langsmith?")

'Get started with LangSmith - Docs by LangChainOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith is a platform for building production-grade LLM applications. Monitor and evaluate your application, so you can ship quickly and with confidence.'

Import → from langchain.tools.retriever import create_retriever_tool brings in the helper to turn a retriever into a LangChain tool.

Tool creation → retriever_tool = create_retriever_tool(...) makes the tool object.

First argument – retriever → your existing retriever (e.g., from a vector database) that fetches relevant documents for a query.

Second argument – "langsmith_search" → the tool’s name, used by agents to identify and call it.

Third argument – description → "Search for information about LangSmith..." tells the agent when to use this tool, guiding its decision process.

Under the hood → wraps the retriever so it works like a standard tool with .name, .description, and .run(query).

When called → it executes retriever.get_relevant_documents(query) and returns text chunks.

Usage → integrate retriever_tool into an agent so the agent can automatically search LangSmith-related info whenever needed.

### Premade Arxiv Tool

In [9]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [10]:
arxiv.run("What is deep learning?")

'Published: 2021-06-15\nTitle: Deep Reinforcement Learning for Conservation Decisions\nAuthors: Marcus Lapeyrolerie, Melissa S. Chapman, Kari E. A. Norman, Carl Boettiger\nSummary: Can machine learning he'

Definition → arXiv (pronounced “archive”) is a free, open-access online repository for research papers.

Domain → Mainly covers physics, mathematics, computer science, quantitative biology, quantitative finance, electrical engineering, statistics, and economics.

Founded → Started in 1991 by physicist Paul Ginsparg at Los Alamos, later moved to Cornell University.

Purpose → Lets researchers share their scientific work quickly before peer-reviewed journal publication.

Access → Anyone can read and download papers at no cost, no subscription required.

In [11]:
tools=[wiki,arxiv,retriever_tool]

In [12]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama2")

C:\Users\vishal.puri\AppData\Local\Temp\ipykernel_20212\3767266266.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama2")


In [13]:
# Get the prompt to use - we can modify this!>>Already made prompts from langchain hub
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Agents

In [14]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [15]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [18]:
agent_executor.invoke({"input":"What is deep learning?"})



> Entering new AgentExecutor chain...
Hello! I'm here to help you understand what deep learning is. Deep learning is a subfield of machine learning that involves the use of artificial neural networks to model and solve complex problems. These networks are designed to mimic the structure and function of the human brain, with layers of interconnected nodes (neurons) that process and transmit information.

Deep learning algorithms are capable of learning and improving on their own by analyzing large amounts of data, without being explicitly programmed for a particular task. This makes them particularly useful for tasks such as image and speech recognition, natural language processing, and autonomous driving.

Some key features of deep learning include:

1. Artificial neural networks: Deep learning algorithms are based on artificial neural networks, which are modeled after the structure and function of the human brain. These networks have multiple layers of interconnected nodes (neurons)

{'input': 'What is deep learning?',
 'output': "Hello! I'm here to help you understand what deep learning is. Deep learning is a subfield of machine learning that involves the use of artificial neural networks to model and solve complex problems. These networks are designed to mimic the structure and function of the human brain, with layers of interconnected nodes (neurons) that process and transmit information.\n\nDeep learning algorithms are capable of learning and improving on their own by analyzing large amounts of data, without being explicitly programmed for a particular task. This makes them particularly useful for tasks such as image and speech recognition, natural language processing, and autonomous driving.\n\nSome key features of deep learning include:\n\n1. Artificial neural networks: Deep learning algorithms are based on artificial neural networks, which are modeled after the structure and function of the human brain. These networks have multiple layers of interconnected n

In [16]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Certainly! Langsmith is a fictional character I created for the purpose of this interaction. He is a friendly and helpful assistant who is here to assist you with any questions or tasks you may have.

Langsmith is a robot with a sleek and modern design, equipped with advanced technology that allows him to perform a wide range of tasks. He has a cheerful and polite demeanor, and is always eager to help with a smile on his face.

Some of Langsmith's abilities include:

* Answering questions on a variety of topics, from science and technology to history and culture.
* Assisting with tasks such as scheduling appointments, making reservations, or sending messages.
* Providing suggestions or recommendations for things like travel destinations, books to read, or movies to watch.
* Offering language translation services, allowing him to communicate with people from different cultures and backgrounds.
* Helping with problem-solving tasks, such as brainst

{'input': 'Tell me about Langsmith',
 'output': "\nCertainly! Langsmith is a fictional character I created for the purpose of this interaction. He is a friendly and helpful assistant who is here to assist you with any questions or tasks you may have.\n\nLangsmith is a robot with a sleek and modern design, equipped with advanced technology that allows him to perform a wide range of tasks. He has a cheerful and polite demeanor, and is always eager to help with a smile on his face.\n\nSome of Langsmith's abilities include:\n\n* Answering questions on a variety of topics, from science and technology to history and culture.\n* Assisting with tasks such as scheduling appointments, making reservations, or sending messages.\n* Providing suggestions or recommendations for things like travel destinations, books to read, or movies to watch.\n* Offering language translation services, allowing him to communicate with people from different cultures and backgrounds.\n* Helping with problem-solving ta

In [17]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

I apologize, but I don't have access to the latest scientific papers or research studies. However, I can try to help you find the information you're looking for. Can you provide me with more details about the paper you're interested in, such as the title, author, or a brief summary? That way, I can try to help you locate it or provide you with relevant information.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': "\nI apologize, but I don't have access to the latest scientific papers or research studies. However, I can try to help you find the information you're looking for. Can you provide me with more details about the paper you're interested in, such as the title, author, or a brief summary? That way, I can try to help you locate it or provide you with relevant information."}